In [1]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from sqlalchemy import create_engine, text, MetaData, Table, Column, Integer, String, Boolean, Float, ForeignKey
# from sys import getsizeof

In [2]:
url = 'https://api.entur.io/realtime/v1/rest/vm?datasetId=KOL'
response = requests.get(url)

In [3]:
import xmltodict, json

xmlAsDict = xmltodict.parse(response.content)
test = json.dumps(xmlAsDict) # '{"e": {"a": ["text", "text"]}}'

VehicleActivities = xmlAsDict["Siri"]["ServiceDelivery"]["VehicleMonitoringDelivery"]["VehicleActivity"]

responseTimestamp = xmlAsDict["Siri"]["ServiceDelivery"]["ResponseTimestamp"]

for i, e in enumerate(VehicleActivities):
    VehicleActivities[i]["ResponseTimestamp"] = responseTimestamp
    

normal = pd.json_normalize(VehicleActivities)

# Sometimes the new data might have arrived before the old data is not valid anymore
# This leads to the same vehicle being in the list twice in the same response
# Sort by RecordedAtTime and drop duplicates
normal = normal.sort_values('RecordedAtTime', ascending=False)
normal = normal.drop_duplicates('MonitoredVehicleJourney.VehicleRef', keep='first')


,RecordedAtTime,ValidUntilTime,ResponseTimestamp,MonitoredVehicleJourney.LineRef,MonitoredVehicleJourney.FramedVehicleJourneyRef.DataFrameRef,MonitoredVehicleJourney.FramedVehicleJourneyRef.DatedVehicleJourneyRef,MonitoredVehicleJourney.VehicleMode,MonitoredVehicleJourney.OperatorRef,MonitoredVehicleJourney.OriginRef,MonitoredVehicleJourney.OriginName.@xml:lang,...,MonitoredVehicleJourney.Velocity,MonitoredVehicleJourney.Delay,MonitoredVehicleJourney.VehicleStatus,MonitoredVehicleJourney.VehicleRef,MonitoredVehicleJourney.MonitoredCall.StopPointRef,MonitoredVehicleJourney.MonitoredCall.StopPointName,MonitoredVehicleJourney.MonitoredCall.VehicleLocationAtStop.Longitude,MonitoredVehicleJourney.MonitoredCall.VehicleLocationAtStop.Latitude,MonitoredVehicleJourney.MonitoredCall.DestinationDisplay,MonitoredVehicleJourney.IsCompleteStopSequence
176,2024-02-02T15:00:38+01:00,2024-02-02T15:00:53+01:00,2024-02-02T15:00:42.201493647+01:00,1004,2024-02-02,KOL:ServiceJourney:1004_231113103225401_2066,bus,309,NSR:Quay:48760,EN,...,22,PT5M43S,inProgress,2206,NSR:Quay:47292,Gamle Madlaveien,5.7064621,58.9593719,Viste Hageby,false
210,2024-02-02T15:00:38+01:00,2024-02-02T15:00:53+01:00,2024-02-02T15:00:42.201493647+01:00,2015,2024-02-02,KOL:ServiceJourney:2015_231113122831085_1005,bus,310,NSR:Quay:103379,EN,...,0,PT0S,inProgress,3011,NSR:Quay:103379,Sandnes bussterminal Ruten (hpl. 14),5.7368173,58.8511711,Lauvvik via Sviland,false
124,2024-02-02T15:00:38+01:00,2024-02-02T15:00:53+01:00,2024-02-02T15:00:42.201493647+01:00,1012,2024-02-02,KOL:ServiceJourney:1012_231113103212420_1036,bus,309,NSR:Quay:48354,EN,...,25,PT1M58S,inProgress,2237,NSR:Quay:44802,Hagakrossen (hpl. A),5.6008945,58.9355077,Sandnes via Stavanger lufthavn,false
25,2024-02-02T15:00:38+01:00,2024-02-02T15:00:53+01:00,2024-02-02T15:00:42.201493647+01:00,2016,2024-02-02,KOL:ServiceJourney:2016_231113103222976_1016,bus,310,NSR:Quay:49438,EN,...,0,PT4M1S,inProgress,9835,NSR:Quay:46562,Molaug (hpl. B),6.3052209,58.8431662,Eikjeskog,false
77,2024-02-02T15:00:38+01:00,2024-02-02T15:00:53+01:00,2024-02-02T15:00:42.201493647+01:00,3001,2024-02-02,KOL:ServiceJourney:3001_231113103214764_2024,bus,315,NSR:Quay:48361,EN,...,23,PT22S,inProgress,5512,NSR:Quay:47201,Løkavannstunet (hpl. B),5.3299008,59.3961901,Vormedal via Haugesund sentrum,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,2024-02-02T14:51:58+01:00,2024-02-02T14:52:13+01:00,2024-02-02T15:00:42.201493647+01:00,3319,2024-02-02,KOL:ServiceJourney:3319_231113103215216_2002,bus,315,NSR:Quay:46156,EN,...,0,NaN,inProgress,5520,NSR:Quay:45488,Veavågen,5.2194908,59.2957077,Veavågen,false
78,2024-02-02T14:51:58+01:00,2024-02-02T14:52:13+01:00,2024-02-02T15:00:42.201493647+01:00,1010,2024-02-02,KOL:ServiceJourney:1010_231113103212012_2018,bus,309,NSR:Quay:47273,EN,...,32,NaN,inProgress,2321,NSR:Quay:46970,Jernbaneveien (hpl. E),5.7337283,58.9659783,Stavanger,false
314,2024-02-02T14:51:52+01:00,2024-02-02T14:52:07+01:00,2024-02-02T15:00:42.201493647+01:00,2693,2024-02-02,KOL:ServiceJourney:2693_231113103217182_1006,bus,310,NSR:Quay:49483,EN,...,62,NaN,inProgress,3043,NSR:Quay:47437,Sveheia,6.005463,58.8462713,Seldal,false
171,2024-02-02T14:51:03+01:00,2024-02-02T14:51:18+01:00,2024-02-02T15:00:42.201493647+01:00,3320,2024-02-02,KOL:ServiceJourney:3320_231113103221037_2002,bus,315,NSR:Quay:46885,EN,...,0,NaN,inProgress,5572,NSR:Quay:49161,Brekke (hpl. B),5.258135,59.2795428,Brekke,false


In [4]:
engine = create_engine('sqlite:///test.db', echo=False)

metadata = MetaData()

columns = ['RecordedAtTime', 'ValidUntilTime', 'MonitoredVehicleJourney.LineRef',
       'MonitoredVehicleJourney.FramedVehicleJourneyRef.DataFrameRef',
       'MonitoredVehicleJourney.FramedVehicleJourneyRef.DatedVehicleJourneyRef',
       'MonitoredVehicleJourney.VehicleMode',
       'MonitoredVehicleJourney.OperatorRef',
       'MonitoredVehicleJourney.OriginRef',
       'MonitoredVehicleJourney.OriginName.@xml:lang',
       'MonitoredVehicleJourney.OriginName.#text',
       'MonitoredVehicleJourney.DestinationRef',
       'MonitoredVehicleJourney.DestinationName.@xml:lang',
       'MonitoredVehicleJourney.DestinationName.#text',
       'MonitoredVehicleJourney.OriginAimedDepartureTime',
       'MonitoredVehicleJourney.DestinationAimedArrivalTime',
       'MonitoredVehicleJourney.Monitored',
       'MonitoredVehicleJourney.DataSource',
       'MonitoredVehicleJourney.VehicleLocation.Longitude',
       'MonitoredVehicleJourney.VehicleLocation.Latitude',
       'MonitoredVehicleJourney.Bearing', 'MonitoredVehicleJourney.Velocity',
       'MonitoredVehicleJourney.Delay',
       'MonitoredVehicleJourney.VehicleStatus',
       'MonitoredVehicleJourney.VehicleRef',
       'MonitoredVehicleJourney.MonitoredCall.StopPointRef',
       'MonitoredVehicleJourney.MonitoredCall.StopPointName',
       'MonitoredVehicleJourney.MonitoredCall.VehicleLocationAtStop.Longitude',
       'MonitoredVehicleJourney.MonitoredCall.VehicleLocationAtStop.Latitude',
       'MonitoredVehicleJourney.MonitoredCall.DestinationDisplay',
       'MonitoredVehicleJourney.IsCompleteStopSequence']

# Create table from columns
new_table = Table(
   'Vehicles', metadata,
   Column('ResponseTimestamp', String, primary_key=True),
   Column('RecordedAtTime', String),
   Column('ValidUntilTime', String),
   Column('MonitoredVehicleJourney.LineRef', Integer),
   Column('MonitoredVehicleJourney.FramedVehicleJourneyRef.DataFrameRef', String),
   Column('MonitoredVehicleJourney.FramedVehicleJourneyRef.DatedVehicleJourneyRef', String),
   Column('MonitoredVehicleJourney.VehicleMode', String),
   Column('MonitoredVehicleJourney.OperatorRef', Integer),
   Column('MonitoredVehicleJourney.OriginRef', String),
   Column('MonitoredVehicleJourney.OriginName.@xml:lang', String),
   Column('MonitoredVehicleJourney.OriginName.#text', String),
   Column('MonitoredVehicleJourney.DestinationRef', String),
   Column('MonitoredVehicleJourney.DestinationName.@xml:lang', String),
   Column('MonitoredVehicleJourney.DestinationName.#text', String),
   Column('MonitoredVehicleJourney.OriginAimedDepartureTime', String),
   Column('MonitoredVehicleJourney.DestinationAimedArrivalTime', String),
   Column('MonitoredVehicleJourney.Monitored', Boolean),
   Column('MonitoredVehicleJourney.DataSource', String),
   Column('MonitoredVehicleJourney.VehicleLocation.Longitude', Float),
   Column('MonitoredVehicleJourney.VehicleLocation.Latitude', Float),
   Column('MonitoredVehicleJourney.Bearing', Float),
   Column('MonitoredVehicleJourney.Velocity', Integer),
   Column('MonitoredVehicleJourney.Delay', String),
   Column('MonitoredVehicleJourney.VehicleStatus', String),
   Column('MonitoredVehicleJourney.VehicleRef', Integer, primary_key=True),
   Column('MonitoredVehicleJourney.MonitoredCall.StopPointRef', String),
   Column('MonitoredVehicleJourney.MonitoredCall.StopPointName', String),
   Column('MonitoredVehicleJourney.MonitoredCall.VehicleLocationAtStop.Longitude', Float),
   Column('MonitoredVehicleJourney.MonitoredCall.VehicleLocationAtStop.Latitude', Float),
   Column('MonitoredVehicleJourney.MonitoredCall.DestinationDisplay', String),
   Column('MonitoredVehicleJourney.IsCompleteStopSequence', Boolean)
)

metadata.create_all(engine)

In [5]:
with engine.connect() as conn:
    able_info = conn.execute(text("PRAGMA table_info(Vehicles);")).fetchall()
    x= conn.execute(text('SELECT * FROM Vehicles WHERE "MonitoredVehicleJourney.VehicleRef"=2352')).fetchall()
x
able_info

[(0, 'ResponseTimestamp', 'VARCHAR', 1, None, 1),
 (1, 'RecordedAtTime', 'VARCHAR', 0, None, 0),
 (2, 'ValidUntilTime', 'VARCHAR', 0, None, 0),
 (3, 'MonitoredVehicleJourney.LineRef', 'INTEGER', 0, None, 0),
 (4, 'MonitoredVehicleJourney.FramedVehicleJourneyRef.DataFrameRef', 'VARCHAR', 0, None, 0),
 (5, 'MonitoredVehicleJourney.FramedVehicleJourneyRef.DatedVehicleJourneyRef', 'VARCHAR', 0, None, 0),
 (6, 'MonitoredVehicleJourney.VehicleMode', 'VARCHAR', 0, None, 0),
 (7, 'MonitoredVehicleJourney.OperatorRef', 'INTEGER', 0, None, 0),
 (8, 'MonitoredVehicleJourney.OriginRef', 'VARCHAR', 0, None, 0),
 (9, 'MonitoredVehicleJourney.OriginName.@xml:lang', 'VARCHAR', 0, None, 0),
 (10, 'MonitoredVehicleJourney.OriginName.#text', 'VARCHAR', 0, None, 0),
 (11, 'MonitoredVehicleJourney.DestinationRef', 'VARCHAR', 0, None, 0),
 (12, 'MonitoredVehicleJourney.DestinationName.@xml:lang', 'VARCHAR', 0, None, 0),
 (13, 'MonitoredVehicleJourney.DestinationName.#text', 'VARCHAR', 0, None, 0),
 (14, 'Mo

In [15]:
with engine.connect() as conn:
    trans = conn.begin()
    try:
        normal.to_sql('Vehicles', conn, if_exists='append', index=False)
        trans.commit()
    except Exception as e:
        print(e)
        trans.rollback()

engine.dispose()

sqlalchemy.exc.IntegrityError('(sqlite3.IntegrityError) UNIQUE constraint failed: Vehicles.ResponseTimestamp, Vehicles.MonitoredVehicleJourney.VehicleRef')

In [ ]:
# import pyarrow.parquet as pq
# pq.write_table(normal, 'example.parquet')
# # load parquet file
# df = pd.read_parquet('test.parquet')
# df